In [1]:
import pandas as pd
import numpy as np
np.random.seed(1337)
from keras import Sequential
from keras.utils import Sequence
from keras.layers import LSTM, Dense, Masking
import numpy as np
import keras
from keras.utils import np_utils
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Embedding, Dense, Input, concatenate, Layer, Lambda, Dropout, Activation
import datetime
from datetime import datetime
from keras.callbacks import ModelCheckpoint, EarlyStopping, Callback, TensorBoard
import tensorflow as tf
import tensorflow_hub as hub

Using TensorFlow backend.


In [16]:
transcript_raw = pd.read_csv('Data/transcript.csv')
transcript_raw.columns = transcript_raw.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
transcript_raw = transcript_raw[transcript_raw.human_transcript.notnull()]
transcript_raw.head()

,unnamed:_0,interview_id,interview_date,record_creation_date,venue,address_name,street,city,state_/_province,postal_code,...,interviewer_relationship_5_to_storyteller_2,language_1,language_2,language_3,keywords_-_fixed_subjects,keywords_-_general,keywords_-_places,deep_speech,human_transcript,google_transcript
9,9,MBY005779,10/2/09 11:30,10/2/09,MobileBooth West (MBY),MobileBooth West,Penrose Public Library,Colorado Springs,CO,80911.0,...,NaN,English,NaN,NaN,Achievements and Awards\n\nArchitecture\n\nChi...,1941 Chevy\n\n1960 Starline\n\nagriculture\n\n...,NaN,i my name is lisande i am sixty eight years ol...,LOUISA SANCHEZ: [00:00:00] My name is Luisa Sa...,my name is Teresa Sanchez I am 68 years old to...
16,16,MBY005843,10/14/09 11:30,10/14/09,MobileBooth West (MBY),MobileBooth West,Penrose Public Library,Colorado Springs,CO,80911.0,...,NaN,English,NaN,NaN,Achievements and Awards\n\nChildren\n\nChristm...,Air Corps\n\ncarols\n\nchurch\n\nColorado Spri...,NaN,my name is simply a le and shut stenia i go by...,Cynthia Lee Anschutz Stenicka:\t[00:00:00] My ...,my name is Cynthia Lee and shoots tehnika I go...
17,17,MBY005844,10/14/09 12:30,10/14/09,MobileBooth West (MBY),MobileBooth West,Penrose Public Library,Colorado Springs,CO,80911.0,...,NaN,English,NaN,NaN,Army\n\nBest Friends\n\nBirth\n\nChanges In Ed...,Angela Gonzales\n\nappearance\n\nbirth of firs...,NaN,okay i'm tones rein twenty nine in today's oct...,Denise Ricks: [00:00:00] Okay. I'm Denise Rick...,okay I'm Denise Rick's I'm 29 today is October...
22,22,MBY005884,10/25/09 12:30,10/25/09,MobileBooth West (MBY),MobileBooth West,204 South Main Street,Wichita,KS,67202.0,...,NaN,English,NaN,NaN,Achievements and Awards\n\nBirth\n\nChanges In...,AFLCIO\n\nBarack Obama\n\nBlind Rights\n\nbrai...,NaN,i mane is highly christine johnson i'm thirty ...,Heidi Christine Johnson: [00:00:00] Hi. My nam...,hi my name is Heidi Christine Johnson I'm 38 y...
23,23,MBY005906,10/31/09 9:30,10/31/09,MobileBooth West (MBY),MobileBooth West,204 South Main Street,Wichita,KS,67202.0,...,NaN,English,NaN,NaN,Birth\n\nBosses\n\nComing Of Age\n\nCommunity ...,cohorts (groups of friends)\n\ncollege\n\ncraf...,NaN,i am the morning my name's carcava i am twenty...,Sarah Culver: [00:00:00] Hi. Good morning. My ...,hi good morning my name is Sarah Culver I'm 24...


In [22]:
transcript_raw.shape

(79, 61)

In [17]:
label_raw = pd.read_csv('Data/labels.csv')
label_raw.columns = label_raw.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
label_raw.head()

,interview_id,interview_date,record_creation_date,venue,address_name,street,city,state_/_province,postal_code,interview_description,...,interviewer_relationship_1_to_storyteller_1,language_1,keywords_-_fixed_subjects,keywords_-_general,label1-fs,label2-fs,label3-fs,label1-g,label2-g,label3-g
0,MBY005668,2009/9/5 11:30,2009/9/5,MobileBooth West (MBY),MobileBooth West,Paonia Public Library,Paonia,CO,81428,Rita Elvia Loya (38) habla con su esposo Omar ...,...,wife,Spanish,Achievements and Awards\n\nChildren\n\nCommuni...,"bilingual\n\nBilingue\n\nChihuahua, Mexico\n\n...",Children,Workday Life,Immigration Stories,social beliefs and practices,ethnicity,memories of former times
1,MBY005712,2009/9/13 17:30,2009/9/13,MobileBooth West (MBY),MobileBooth West,Paonia Public Library,Paonia,CO,81428,Cecilio Montes (46) habla con esposa Bertha Ja...,...,esposa,Spanish,Anniversaries\n\nChildren\n\nCommunity Busines...,accidents\n\nbaile\n\ncar accident\n\ncemetery...,Children,Workday Life,Immigration Stories,memories of former times,spouse,family tradition
2,MBY005713,2009/9/14 9:30,2009/9/14,MobileBooth West (MBY),MobileBooth West,Paonia Public Library,Paonia,CO,81428,Alice talks about childhood. Melissa remember...,...,grandmother,English,Birth\n\nBurials\n\nCancer\n\nChildcare\n\nChi...,alcohol\n\nbirth of first child\n\nchemotherap...,Parents,Children,Marriage,memories of growing up,memories of former times,family tradition
3,MBY005730,2009/9/18 12:30,2009/9/18,MobileBooth West (MBY),MobileBooth West,Paonia Public Library,Paonia,CO,81428,Adolph Carranza Jr. (64) talks with friend Ele...,...,friend,NaN,Changes In Education\n\nCommunity Businesses\n...,anecdotes (humorous but true stories)\n\nappea...,Parents,Schools,Workday Life,memories of growing up,social beliefs and practices,ethnicity
4,MBY005764,2009/9/28 10:30,2009/9/28,MobileBooth West (MBY),MobileBooth West,Public Library,Colorado Springs,CO,80911,Rosana interviews her friend Alberta about her...,...,friend,NaN,Education\n\nAchievements and Awards\n\nArmy\n...,advocacy\n\nadvocate\n\ncollege\n\ndevelopment...,Schools,Children,Workday Life,memories of former times,school day memories,spouse


In [27]:
train_raw = transcript_raw.merge(label_raw, on='interview_id')
train_raw.head()

,unnamed:_0,interview_id,interview_date_x,record_creation_date_x,venue_x,address_name_x,street_x,city_x,state_/_province_x,postal_code_x,...,interviewer_relationship_1_to_storyteller_1_y,language_1_y,keywords_-_fixed_subjects_y,keywords_-_general_y,label1-fs,label2-fs,label3-fs,label1-g,label2-g,label3-g
0,9,MBY005779,10/2/09 11:30,10/2/09,MobileBooth West (MBY),MobileBooth West,Penrose Public Library,Colorado Springs,CO,80911.0,...,husband,English,Achievements and Awards\n\nArchitecture\n\nChi...,1941 Chevy\n\n1960 Starline\n\nagriculture\n\n...,Parents,Schools,Children,memories of growing up,ethnicity,anecdotes (humorous but true stories)
1,16,MBY005843,10/14/09 11:30,10/14/09,MobileBooth West (MBY),MobileBooth West,Penrose Public Library,Colorado Springs,CO,80911.0,...,mother,English,Achievements and Awards\n\nChildren\n\nChristm...,Air Corps\n\ncarols\n\nchurch\n\nColorado Spri...,Parents,Children,Workday Life,memories of growing up,ethnicity,memories of former times
2,17,MBY005844,10/14/09 12:30,10/14/09,MobileBooth West (MBY),MobileBooth West,Penrose Public Library,Colorado Springs,CO,80911.0,...,father,English,Army\n\nBest Friends\n\nBirth\n\nChanges In Ed...,Angela Gonzales\n\nappearance\n\nbirth of firs...,Parents,Children,Workday Life,memories of growing up,social beliefs and practices,ethnicity
3,22,MBY005884,10/25/09 12:30,10/25/09,MobileBooth West (MBY),MobileBooth West,204 South Main Street,Wichita,KS,67202.0,...,friend,English,Achievements and Awards\n\nBirth\n\nChanges In...,AFLCIO\n\nBarack Obama\n\nBlind Rights\n\nbrai...,Schools,Workday Life,Immigration Stories,memories of growing up,social beliefs and practices,memories of former times
4,23,MBY005906,10/31/09 9:30,10/31/09,MobileBooth West (MBY),MobileBooth West,204 South Main Street,Wichita,KS,67202.0,...,interviewee,English,Birth\n\nBosses\n\nComing Of Age\n\nCommunity ...,cohorts (groups of friends)\n\ncollege\n\ncraf...,Parents,Schools,Workday Life,memories of growing up,social beliefs and practices,ethnicity


In [28]:
train_raw.shape

(57, 84)

In [29]:
train_raw = train_raw[['human_transcript', 'label1-g']]
train_raw.reset_index(inplace=True, drop=True)
train_raw.head()

,human_transcript,label1-g
0,LOUISA SANCHEZ: [00:00:00] My name is Luisa Sa...,memories of growing up
1,Cynthia Lee Anschutz Stenicka:\t[00:00:00] My ...,memories of growing up
2,Denise Ricks: [00:00:00] Okay. I'm Denise Rick...,memories of growing up
3,Heidi Christine Johnson: [00:00:00] Hi. My nam...,memories of growing up
4,Sarah Culver: [00:00:00] Hi. Good morning. My ...,memories of growing up


In [30]:
for l in np.unique(train_raw['label1-g']):
  print(l)

ethnicity
memories of growing up
school day memories
social beliefs and practices


In [32]:
train_raw=train_raw.rename(columns = {'human_transcript':'text', 'label1-g':'label'})
train_raw.head()

,text,label
0,LOUISA SANCHEZ: [00:00:00] My name is Luisa Sa...,memories of growing up
1,Cynthia Lee Anschutz Stenicka:\t[00:00:00] My ...,memories of growing up
2,Denise Ricks: [00:00:00] Okay. I'm Denise Rick...,memories of growing up
3,Heidi Christine Johnson: [00:00:00] Hi. My nam...,memories of growing up
4,Sarah Culver: [00:00:00] Hi. Good morning. My ...,memories of growing up


In [33]:
from sklearn.preprocessing import LabelEncoder

LE = LabelEncoder()
train_raw['label'] = LE.fit_transform(train_raw['label'])
train_raw.head()

,text,label
0,LOUISA SANCHEZ: [00:00:00] My name is Luisa Sa...,1
1,Cynthia Lee Anschutz Stenicka:\t[00:00:00] My ...,1
2,Denise Ricks: [00:00:00] Okay. I'm Denise Rick...,1
3,Heidi Christine Johnson: [00:00:00] Hi. My nam...,1
4,Sarah Culver: [00:00:00] Hi. Good morning. My ...,1


In [34]:
len(np.unique(train_raw['label']))

4

In [35]:
train = train_raw.copy()

In [36]:
train = train.reindex(np.random.permutation(train.index))
train.head()

,text,label
35,Tomas:\t[00:00:00] My name is Tomas Garduno. I...,1
30,Natalie Romero:\t[00:00:00] Hi. My name is Nat...,1
0,LOUISA SANCHEZ: [00:00:00] My name is Luisa Sa...,1
37,Interviewer:\t[00:00:00] Go.Jose Armas:\tMy na...,3
10,Bolivar Moyano Fraga: [00:00:00] My name is Bo...,1


In [37]:
import re
def clean_txt(text):
  text = re.sub("'", "",text)
  text=re.sub("(\\W)+"," ",text)    
  return text

In [38]:
train['text']  = train.text.apply(clean_txt)
train.head()

,text,label
35,Tomas 00 00 00 My name is Tomas Garduno Im 31 ...,1
30,Natalie Romero 00 00 00 Hi My name is Natalie ...,1
0,LOUISA SANCHEZ 00 00 00 My name is Luisa Sanch...,1
37,Interviewer 00 00 00 Go Jose Armas My name is ...,3
10,Bolivar Moyano Fraga 00 00 00 My name is Boliv...,1


In [39]:
from sklearn.model_selection import train_test_split
train, val = train_test_split(train, test_size=0.2, random_state=35)
train.head()

,text,label
14,John Rivera Sedlar 00 00 00 Hi my name is John...,3
47,Santos Martinez 00 00 00 Im Santos Martinez Im...,3
39,Eloise Serna 00 00 00 My name is Eloise Serna ...,1
26,Naomi Guerrero 00 00 00 Hello My name is Naomi...,1
46,Sabina Maria Zuniga Varela 00 00 00 My name is...,1


In [40]:
train.reset_index(drop=True, inplace=True)
train.head(2)

,text,label
0,John Rivera Sedlar 00 00 00 Hi my name is John...,3
1,Santos Martinez 00 00 00 Im Santos Martinez Im...,3


In [41]:
val.reset_index(drop=True, inplace=True)
val.head(2)

,text,label
0,BARBARA ASHUD 00 00 00 My name is Barbra Ashud...,1
1,FEMALE_1 00 00 00 Good afternoon My name is Ni...,1


In [42]:
val.shape, train.shape

((12, 2), (45, 2))